# Analysis of NYC public schools results in ELA and math grades 6-8

<span style="color: red;">**If kernel can't connect to server again run command:**
*netsh winsock reset*<span>

### Processing data by schools

<a id="TOC"></a> 
## Table of Contents
1. [Data sources and definitions](#data)
2. [Research questions](#questions)
2. [Analysis of test results by middle schools](#analysis)
    1. [Imports: modules](#modules)
    3. [Read and prepare data](#read)
    4. [Getting the baseline change in tests results - citywide change](#citywide)
    5. [Getting the test results for middle schools and calculate comparison indicator by school](#middle) 
        1. [Best middle schools by math](#best)
        2. [Create dataframe with average 2013-2023 math and ELA test results for all middle school grades](#ten)
        3. [Create dataframe with average 2019-2023 (last 3 tests) math and ela test results for all middle school grades](#three)    
    6.[Create final dataframe with data for mapping](#final)
        1. [Adding school status (citywide, boroughwide) and the diversity data to the dataframe with all tests resuls](#status)
        2. [Matching the schools names from GeoJSON schools location file and the results dataframe and merging](#match)
        3. [Adding history ELA/math results, demographic data as plots to the geodata frame and saving into GeoJSON file](#plots)

<a id="data"></a> 
### Data sources and definitions

#### Data:
1. New York City grades 3-8 New York State English Language Arts and Math State Tests results 2013-2023:<br>https://data.cityofnewyork.us/
<br>New York City grades 3-8 New York State English Language Arts and Math State Tests results 2018-2025: <br>https://infohub.nyced.org/reports/academics/test-results
2. New York City schools demographic data:<br>https://data.cityofnewyork.us/Education/2017-18-2021-22-Demographic-Snapshot/c7ru-d68s/about_data
2. NYS schools locations:<br>
https://data.gis.ny.gov/maps/b6c624c740e4476689aa60fdc4aacb8f/about
3. Citywide or Boroughwide status:
<br>https://www.nycschoolhelp.com/borowide-citywide-middle-schools

#### Definitions of Performance Levels for the 2023 Grades 3-8 English Language Arts and Mathematics Tests  

**NYS Level 1**: Students performing at this level are below proficient in standards for their grade. They may demonstrate limited knowledge, skills, and practices embodied by the Learning Standards that are considered insufficient for the expectations at this grade. 

**NYS Level 2**: Students performing at this level are partially proficient in standards for their grade. They demonstrate knowledge, skills, and practices embodied by the Learning Standards that are considered partial but insufficient for the expectations at this grade. Students performing at Level 2 are considered on track to meet current New York high school graduation requirements but are not yet proficient in Learning Standards at this grade. 

**NYS Level 3**: Students performing at this level are proficient in standards for their grade. They demonstrate knowledge, skills, and practices embodied by the Learning Standards that are considered sufficient for the expectations at this grade.  

**NYS Level 4**: Students performing at this level excel in standards for their grade. They demonstrate knowledge, skills, and practices embodied by the Learning Standards that are considered more than sufficient for the expectations at this grade.  

*Source: NYSED, 2023, https://www.p12.nysed.gov/irs/ela-math/2023/ela-math-score-ranges-performance-levels-2023.pdf*

<a id="questions"></a> 
### Questions
*1. How to compare the schools?*
<br>In this analysis, we choose the sum of shares of students with level 4 test resulsts in state math and ELA test as comparison variable. The sum can be between 0 and 2. This indicator is selected to cover both subjects.
Alternatively, the indicator can be sum of shares of students with levels 3+4 test results in math and ELA. The notebook would be needed to changed accordingly.
<br><br>
*2. How the test results changed?*
<br>Compare last year test results in a school with the school 10-year average as percentage of average:
<br> school_change = (school_current_year - school_10year_average)
<br> citywide_change = (city_current_year - city_10year_average)
<br><br>
*3. How good the school is?* 
<br>Last three testing period results (2023, 2024, 2025) are different for some schools: due to COVID disruptions, testing procedures changes, in Destrict 15 due to admission rules changed. Therefore average 10 years scores do not reflect well schools situation now. Results for these 3 last testing years are taken instead.
<br><br>
*3. Does the school have a citywide or boroughwide priority as opposed to disctrict priority (here we have unchecked data for Brooklyn only)?*
<br>
*4. Diversity?*
<br>
*5. School size?*

#### About this notebook

- This notebook '*1._NYC_data_processing_by_schools.ipynb*' contains the steps for the processing data on state testing of NYC public middle schools. 
- The notebook '*2._NYC_ELA_math_data processing_by_districts.ipynb*' contains steps to process district-wide data for NYC public middle schools.
- The notebook '*3._Generating_NYC_map_by_public_schools.ipynb*' contains code to generate the maps from the processed data.
- The map is available at: https://nycmsmap.netlify.app.

<a id="analysis"></a> 
### Analysis of test results by middle schools

<a id="modules"></a> 
#### Imports: modules

In [1]:
# Appending the path to 'utils' module

import sys

parent_dir = 'C:\\GITHUB\\NY_schools_maps\\notebooks'
sys.path.append(parent_dir)

In [2]:
import os
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import base64
from io import BytesIO
import math
from tqdm import tqdm
from utils import match_name, create_plot, process_schools, create_chart

pd.set_option('display.float_format', '{:.3f}'.format)

c:\GITHUB\NY_schools_maps\.conda\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
#To reload 'uitls' module if something changed

import utils
from importlib import reload
reload(utils)

Processing the information on citywide or open to Brooklyn borough residents middle schools from the *nyc school help* webpage into a csv file for reuse.

<a id="read"></a> 
#### Read data

In [3]:
basePath = r"G:\My Drive\Kids\NYC_schools_mapped"
dataFolder = r"raw_data"
outputFolder = r"processed_data"

In [6]:
## Read data by schools

#Read math results
fileName_math = "school-math-results-2013-2023-(public).xlsx"
mathPath = os.path.join(basePath,dataFolder,fileName_math)
print(mathPath)
sheetName_math = "All"
mathResultsDF = pd.read_excel(mathPath, sheetName_math)

#Read math results
fileName_math2025 = "school-math-results-2018-2025-public.xlsx"
mathPath2 = os.path.join(basePath,dataFolder,fileName_math2025)
print(mathPath2)
sheetName_math2 = "Math - All"
math2025DF = pd.read_excel(mathPath2, sheetName_math2)

#Read ELA results
fileName_ELA = "school-ela-results-2013-2023-(public).xlsx"
ELAPath = os.path.join(basePath, dataFolder, fileName_ELA)
print(ELAPath)
sheetName_ELA = "All"
ELAResultsDF = pd.read_excel(ELAPath, sheetName_ELA)

#Read ELA results
fileName_ELA2025 = "school-ela-results-2018-2025-public.xlsx"
ELAPath2 = os.path.join(basePath, dataFolder, fileName_ELA2025)
print(ELAPath2)
sheetName_ELA2 = "ELA - All"
ELA2025DF = pd.read_excel(ELAPath2, sheetName_ELA2)

#Read demographic file
fileName_demog = "demographic-snapshot-2018-19-to-2022-23-(public).xlsx"
demogPath = os.path.join(basePath, dataFolder, fileName_demog)
print(demogPath)
sheetName_demog = "School"
demogData = pd.read_excel(demogPath, sheetName_demog)

#Read school status file
fileName_status = "cityBoroughWideschools.csv"
statusPath = os.path.join(basePath, dataFolder, fileName_status)
print(statusPath)
statusData = pd.read_csv(statusPath)

G:\My Drive\Kids\NYC_schools_mapped\raw_data\school-math-results-2013-2023-(public).xlsx
G:\My Drive\Kids\NYC_schools_mapped\raw_data\school-math-results-2018-2025-public.xlsx
G:\My Drive\Kids\NYC_schools_mapped\raw_data\school-ela-results-2013-2023-(public).xlsx
G:\My Drive\Kids\NYC_schools_mapped\raw_data\school-ela-results-2018-2025-public.xlsx
G:\My Drive\Kids\NYC_schools_mapped\raw_data\demographic-snapshot-2018-19-to-2022-23-(public).xlsx
G:\My Drive\Kids\NYC_schools_mapped\raw_data\cityBoroughWideschools.csv


In [7]:
math2025DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28439 entries, 0 to 28438
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   DBN               28439 non-null  object
 1   School Name       28439 non-null  object
 2   Grade             28439 non-null  object
 3   Year              28439 non-null  int64 
 4   Category          28439 non-null  object
 5   Number Tested     28439 non-null  int64 
 6   Mean Scale Score  28439 non-null  object
 7   # Level 1         28439 non-null  object
 8   % Level 1         28439 non-null  object
 9   # Level 2         28439 non-null  object
 10  % Level 2         28439 non-null  object
 11  # Level 3         28439 non-null  object
 12  % Level 3         28439 non-null  object
 13  # Level 4         28439 non-null  object
 14  % Level 4         28439 non-null  object
 15  # Level 3+4       28439 non-null  object
 16  % Level 3+4       28439 non-null  object
dtypes: int64(2),

In [8]:
ELA2025DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29012 entries, 0 to 29011
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   DBN               29012 non-null  object
 1   School Name       29012 non-null  object
 2   Grade             29012 non-null  object
 3   Year              29012 non-null  int64 
 4   Category          29012 non-null  object
 5   Number Tested     29012 non-null  int64 
 6   Mean Scale Score  29012 non-null  object
 7   # Level 1         29012 non-null  object
 8   % Level 1         29012 non-null  object
 9   # Level 2         29012 non-null  object
 10  % Level 2         29012 non-null  object
 11  # Level 3         29012 non-null  object
 12  % Level 3         29012 non-null  object
 13  # Level 4         29012 non-null  object
 14  % Level 4         29012 non-null  object
 15  # Level 3+4       29012 non-null  object
 16  % Level 3+4       29012 non-null  object
dtypes: int64(2),

In [9]:
colToConvert = ['Mean Scale Score',
     'Grade',                             
     '# Level 1',
     '% Level 1',
     '# Level 2',
     '% Level 2',
     '# Level 3',
     '% Level 3',
     '# Level 4',
     '% Level 4',
     '# Level 3+4',
     '% Level 3+4']
math2025DF[colToConvert] = math2025DF[colToConvert].apply(pd.to_numeric, errors = 'coerce')
math2025DF.info()
ELA2025DF[colToConvert] = ELA2025DF[colToConvert].apply(pd.to_numeric, errors = 'coerce')
ELA2025DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28439 entries, 0 to 28438
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   DBN               28439 non-null  object 
 1   School Name       28439 non-null  object 
 2   Grade             21774 non-null  float64
 3   Year              28439 non-null  int64  
 4   Category          28439 non-null  object 
 5   Number Tested     28439 non-null  int64  
 6   Mean Scale Score  28023 non-null  float64
 7   # Level 1         28023 non-null  float64
 8   % Level 1         28023 non-null  float64
 9   # Level 2         28023 non-null  float64
 10  % Level 2         28023 non-null  float64
 11  # Level 3         28023 non-null  float64
 12  % Level 3         28023 non-null  float64
 13  # Level 4         28023 non-null  float64
 14  % Level 4         28023 non-null  float64
 15  # Level 3+4       28023 non-null  float64
 16  % Level 3+4       28023 non-null  float6

In [10]:
math2025DF2 = math2025DF[math2025DF['Year'].isin([2024, 2025])]
math2025DF2.head(20)

,DBN,School Name,Grade,Year,Category,Number Tested,Mean Scale Score,# Level 1,% Level 1,# Level 2,% Level 2,# Level 3,% Level 3,# Level 4,% Level 4,# Level 3+4,% Level 3+4
0,01M015,P.S. 015 ROBERTO CLEMENTE,3.000,2025,All Students,21,459.048,1.000,4.762,8.000,38.095,7.000,33.333,5.000,23.810,12.000,57.143
1,01M015,P.S. 015 ROBERTO CLEMENTE,4.000,2025,All Students,22,470.727,1.000,4.545,3.000,13.636,9.000,40.909,9.000,40.909,18.000,81.818
2,01M015,P.S. 015 ROBERTO CLEMENTE,5.000,2025,All Students,29,455.483,9.000,31.034,7.000,24.138,5.000,17.241,8.000,27.586,13.000,44.828
3,01M015,P.S. 015 ROBERTO CLEMENTE,NaN,2025,All Students,72,461.181,11.000,15.278,18.000,25.000,21.000,29.167,22.000,30.556,43.000,59.722
4,01M015,P.S. 015 ROBERTO CLEMENTE,3.000,2024,All Students,23,448.478,5.000,21.739,7.000,30.435,7.000,30.435,4.000,17.391,11.000,47.826
5,01M015,P.S. 015 ROBERTO CLEMENTE,4.000,2024,All Students,26,463.192,4.000,15.385,3.000,11.538,12.000,46.154,7.000,26.923,19.000,73.077
6,01M015,P.S. 015 ROBERTO CLEMENTE,5.000,2024,All Students,15,450.733,2.000,13.333,4.000,26.667,8.000,53.333,1.000,6.667,9.000,60.000
7,01M015,P.S. 015 ROBERTO CLEMENTE,NaN,2024,All Students,64,454.984,11.000,17.188,14.000,21.875,27.000,42.188,12.000,18.750,39.000,60.938
25,01M020,P.S. 020 ANNA SILVER,3.000,2025,All Students,42,432.548,15.000,35.714,15.000,35.714,12.000,28.571,0.000,0.000,12.000,28.571
26,01M020,P.S. 020 ANNA SILVER,4.000,2025,All Students,46,446.565,20.000,43.478,8.000,17.391,10.000,21.739,8.000,17.391,18.000,39.130


In [12]:
ELA2025DF2 = ELA2025DF[ELA2025DF['Year'].isin([2024, 2025])]
ELA2025DF2.head(20)

,DBN,School Name,Grade,Year,Category,Number Tested,Mean Scale Score,# Level 1,% Level 1,# Level 2,% Level 2,# Level 3,% Level 3,# Level 4,% Level 4,# Level 3+4,% Level 3+4
0,01M015,P.S. 015 ROBERTO CLEMENTE,3.000,2025,All Students,21,448.190,4.000,19.048,8.000,38.095,4.000,19.048,5.000,23.810,9.000,42.857
1,01M015,P.S. 015 ROBERTO CLEMENTE,4.000,2025,All Students,22,461.000,1.000,4.545,5.000,22.727,9.000,40.909,7.000,31.818,16.000,72.727
2,01M015,P.S. 015 ROBERTO CLEMENTE,5.000,2025,All Students,27,458.815,2.000,7.407,3.000,11.111,15.000,55.556,7.000,25.926,22.000,81.481
3,01M015,P.S. 015 ROBERTO CLEMENTE,NaN,2025,All Students,70,456.314,7.000,10.000,16.000,22.857,28.000,40.000,19.000,27.143,47.000,67.143
4,01M015,P.S. 015 ROBERTO CLEMENTE,3.000,2024,All Students,22,445.455,6.000,27.273,8.000,36.364,4.000,18.182,4.000,18.182,8.000,36.364
5,01M015,P.S. 015 ROBERTO CLEMENTE,4.000,2024,All Students,26,457.615,2.000,7.692,6.000,23.077,8.000,30.769,10.000,38.462,18.000,69.231
6,01M015,P.S. 015 ROBERTO CLEMENTE,5.000,2024,All Students,16,441.625,2.000,12.500,11.000,68.750,2.000,12.500,1.000,6.250,3.000,18.750
7,01M015,P.S. 015 ROBERTO CLEMENTE,NaN,2024,All Students,64,449.438,10.000,15.625,25.000,39.062,14.000,21.875,15.000,23.438,29.000,45.312
25,01M020,P.S. 020 ANNA SILVER,3.000,2025,All Students,35,434.457,17.000,48.571,8.000,22.857,9.000,25.714,1.000,2.857,10.000,28.571
26,01M020,P.S. 020 ANNA SILVER,4.000,2025,All Students,41,445.902,15.000,36.585,7.000,17.073,11.000,26.829,8.000,19.512,19.000,46.341


In [14]:
mathResultsDF = pd.concat([mathResultsDF, math2025DF2], ignore_index = True)
mathResultsDF.head(50)

,DBN,School Name,Grade,Year,Category,Number Tested,Mean Scale Score,# Level 1,% Level 1,# Level 2,% Level 2,# Level 3,% Level 3,# Level 4,% Level 4,# Level 3+4,% Level 3+4
0,01M015,P.S. 015 ROBERTO CLEMENTE,3,2023,All Students,27,447,6,22.222,9,33.333,7,25.926,5,18.519,12,44.444
1,01M015,P.S. 015 ROBERTO CLEMENTE,4,2023,All Students,23,444.870,7,30.435,3,13.043,12,52.174,1,4.348,13,56.522
2,01M015,P.S. 015 ROBERTO CLEMENTE,5,2023,All Students,30,431.867,14,46.667,11,36.667,5,16.667,0,0,5,16.667
3,01M015,P.S. 015 ROBERTO CLEMENTE,6,2023,All Students,1,s,s,s,s,s,s,s,s,s,s,s
4,01M015,P.S. 015 ROBERTO CLEMENTE,All Grades,2023,All Students,81,s,s,s,s,s,s,s,s,s,s,s
5,01M015,P.S. 015 ROBERTO CLEMENTE,3,2022,All Students,21,582.857,12,57.143,4,19.048,5,23.810,0,0,5,23.810
6,01M015,P.S. 015 ROBERTO CLEMENTE,4,2022,All Students,30,590.933,10,33.333,13,43.333,5,16.667,2,6.667,7,23.333
7,01M015,P.S. 015 ROBERTO CLEMENTE,5,2022,All Students,24,589.250,15,62.500,1,4.167,5,20.833,3,12.500,8,33.333
8,01M015,P.S. 015 ROBERTO CLEMENTE,All Grades,2022,All Students,75,588.133,37,49.333,18,24,15,20,5,6.667,20,26.667
9,01M015,P.S. 015 ROBERTO CLEMENTE,3,2019,All Students,26,606.346,4,15.385,4,15.385,8,30.769,10,38.462,18,69.231


In [15]:
ELAResultsDF = pd.concat([ELAResultsDF, ELA2025DF2], ignore_index = True)
ELAResultsDF.tail(20)

,DBN,School Name,Grade,Year,Category,Number Tested,Mean Scale Score,# Level 1,% Level 1,# Level 2,% Level 2,# Level 3,% Level 3,# Level 4,% Level 4,# Level 3+4,% Level 3+4
52102,32K384,P.S. /I.S. 384 FRANCES E. CARTER,6.000,2024,All Students,27,436.000,11.000,40.741,7.000,25.926,8.000,29.630,1.000,3.704,9.000,33.333
52103,32K384,P.S. /I.S. 384 FRANCES E. CARTER,7.000,2024,All Students,22,436.227,7.000,31.818,10.000,45.455,5.000,22.727,0.000,0.000,5.000,22.727
52104,32K384,P.S. /I.S. 384 FRANCES E. CARTER,8.000,2024,All Students,40,437.725,11.000,27.500,18.000,45.000,6.000,15.000,5.000,12.500,11.000,27.500
52105,32K384,P.S. /I.S. 384 FRANCES E. CARTER,NaN,2024,All Students,176,436.398,65.000,36.932,66.000,37.500,32.000,18.182,13.000,7.386,45.000,25.568
52106,32K554,ALL CITY LEADERSHIP SECONDARY SCHOOL,6.000,2025,All Students,57,464.947,1.000,1.754,10.000,17.544,19.000,33.333,27.000,47.368,46.000,80.702
52107,32K554,ALL CITY LEADERSHIP SECONDARY SCHOOL,7.000,2025,All Students,58,473.466,1.000,1.724,3.000,5.172,14.000,24.138,40.000,68.966,54.000,93.103
52108,32K554,ALL CITY LEADERSHIP SECONDARY SCHOOL,8.000,2025,All Students,59,463.424,2.000,3.390,12.000,20.339,21.000,35.593,24.000,40.678,45.000,76.271
52109,32K554,ALL CITY LEADERSHIP SECONDARY SCHOOL,NaN,2025,All Students,174,467.270,4.000,2.299,25.000,14.368,54.000,31.034,91.000,52.299,145.000,83.333
52110,32K554,ALL CITY LEADERSHIP SECONDARY SCHOOL,6.000,2024,All Students,54,469.519,1.000,1.852,4.000,7.407,14.000,25.926,35.000,64.815,49.000,90.741
52111,32K554,ALL CITY LEADERSHIP SECONDARY SCHOOL,7.000,2024,All Students,55,462.800,3.000,5.455,7.000,12.727,24.000,43.636,21.000,38.182,45.000,81.818


In [16]:
# Initializing the list of subjects to use throughout the notebook
subjects = ['Math', 'ELA'] 

In [17]:
# For convinience of future analysis, adding the data tables into dictionnairy by subjects
resultsDFs = {'Math': mathResultsDF, 'ELA': ELAResultsDF}

In [18]:
for subject in subjects:
    resultsDF = resultsDFs[subject]
    resultsDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60996 entries, 0 to 60995
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   DBN               60996 non-null  object
 1   School Name       60996 non-null  object
 2   Grade             56500 non-null  object
 3   Year              60996 non-null  int64 
 4   Category          60996 non-null  object
 5   Number Tested     60996 non-null  int64 
 6   Mean Scale Score  60686 non-null  object
 7   # Level 1         60686 non-null  object
 8   % Level 1         60686 non-null  object
 9   # Level 2         60686 non-null  object
 10  % Level 2         60686 non-null  object
 11  # Level 3         60686 non-null  object
 12  % Level 3         60686 non-null  object
 13  # Level 4         60686 non-null  object
 14  % Level 4         60686 non-null  object
 15  # Level 3+4       60686 non-null  object
 16  % Level 3+4       60686 non-null  object
dtypes: int64(2),

In [19]:
# resultsDF.info() showed that most of the columns are objects instead of numbers and 
# needed to be converted
for subject in subjects:
    resultsDF = resultsDFs[subject]
    resultsDF_colToConvert = ['Mean Scale Score',
     'Grade',                             
     '# Level 1',
     '% Level 1',
     '# Level 2',
     '% Level 2',
     '# Level 3',
     '% Level 3',
     '# Level 4',
     '% Level 4',
     '# Level 3+4',
     '% Level 3+4']
    resultsDF[resultsDF_colToConvert] = resultsDF[resultsDF_colToConvert].apply(pd.to_numeric, errors = 'coerce')
    resultsDF.info()
    print(len(resultsDF))
    
del resultsDF

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60996 entries, 0 to 60995
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   DBN               60996 non-null  object 
 1   School Name       60996 non-null  object 
 2   Grade             46755 non-null  float64
 3   Year              60996 non-null  int64  
 4   Category          60996 non-null  object 
 5   Number Tested     60996 non-null  int64  
 6   Mean Scale Score  60327 non-null  float64
 7   # Level 1         60327 non-null  float64
 8   % Level 1         60327 non-null  float64
 9   # Level 2         60327 non-null  float64
 10  % Level 2         60327 non-null  float64
 11  # Level 3         60327 non-null  float64
 12  % Level 3         60327 non-null  float64
 13  # Level 4         60327 non-null  float64
 14  % Level 4         60327 non-null  float64
 15  # Level 3+4       60327 non-null  float64
 16  % Level 3+4       60327 non-null  float6

In [21]:
## Read citywide data

# Read math results
fileName_cityMath = "citywide-math-results-2013-2025-combined.xlsx"
cityMathPath = os.path.join(basePath,dataFolder,fileName_cityMath)
print(cityMathPath)
sheetName_cityMath = "Math - All"
cityMathDF = pd.read_excel(cityMathPath, sheetName_cityMath)

#Read ELA results
fileName_cityELA = "citywide-ela-results-2013-2025-combined.xlsx"
cityELAPath = os.path.join(basePath, dataFolder, fileName_cityELA)
print(cityELAPath)
sheetName_cityELA = "ELA - All"
cityELADF = pd.read_excel(cityELAPath, sheetName_cityELA)

G:\My Drive\Kids\NYC_schools_mapped\raw_data\citywide-math-results-2013-2025-combined.xlsx
G:\My Drive\Kids\NYC_schools_mapped\raw_data\citywide-ela-results-2013-2025-combined.xlsx


In [22]:
# Dictionnary for citywide results
cityResultsDFs = {'Math': cityMathDF, 'ELA': cityELADF}

In [23]:
# Checking columns types
cityELADF.info()
cityMathDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Grade             77 non-null     object 
 1   Year              77 non-null     int64  
 2   Category          77 non-null     object 
 3   Number Tested     77 non-null     int64  
 4   Mean Scale Score  77 non-null     float64
 5   # Level 1         77 non-null     int64  
 6   % Level 1         77 non-null     float64
 7   # Level 2         77 non-null     int64  
 8   % Level 2         77 non-null     float64
 9   # Level 3         77 non-null     int64  
 10  % Level 3         77 non-null     float64
 11  # Level 4         77 non-null     int64  
 12  % Level 4         77 non-null     float64
 13  # Level 3+4       77 non-null     int64  
 14  % Level 3+4       77 non-null     float64
dtypes: float64(6), int64(7), object(2)
memory usage: 9.2+ KB
<class 'pandas.core.frame.DataFrame'

In [24]:
# 'Grade' column in citywide data tables is object, convert to numeric
for subject in subjects:
    resultsDF = cityResultsDFs[subject]
    resultsDF['Grade'] = resultsDF['Grade'].apply(pd.to_numeric, errors = 'coerce')
    resultsDF.info()
    print(len(resultsDF))
    
del resultsDF

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Grade             66 non-null     float64
 1   Year              77 non-null     int64  
 2   Category          77 non-null     object 
 3   Number Tested     77 non-null     int64  
 4   Mean Scale Score  77 non-null     float64
 5   # Level 1         77 non-null     int64  
 6   % Level 1         77 non-null     float64
 7   # Level 2         77 non-null     int64  
 8   % Level 2         77 non-null     float64
 9   # Level 3         77 non-null     int64  
 10  % Level 3         77 non-null     float64
 11  # Level 4         77 non-null     int64  
 12  % Level 4         77 non-null     float64
 13  # Level 3+4       77 non-null     int64  
 14  % Level 3+4       77 non-null     float64
dtypes: float64(7), int64(7), object(1)
memory usage: 9.2+ KB
77
<class 'pandas.core.frame.DataFra

<a id="citywide"></a> 
### Getting the baseline change in tests results - citywide change

#### Prepare citywide dataframe with only middle school tests results (grades 6-8)

In [25]:
# Select middle school grades results from the citywide dataframes with math and ELA tests results by year
# and calculate percentages of results of each level
resultsMS_Norm = {}

for subject in subjects:
        
    resultsDF = cityResultsDFs[subject]
    
    # Dataframe with only grades 6-8 results (middle schools and K-8) by years
    resultsMS = resultsDF[(resultsDF['Grade'] >= 6)&(resultsDF['Grade'] <= 8)]
    
    # Dataframe with results grouped by years
    resultsMS = resultsMS.groupby('Year')[['# Level 1','# Level 2','# Level 3','# Level 4']].sum()
    
    # Change column names to include subject
    resultsMS.columns = [f'Level 1 {subject}',f'Level 2 {subject}',f'Level 3 {subject}',f'Level 4 {subject}']
    
    # Dataframe for middle schools by years with normalized values
    resultsMS_Norm[subject] = resultsMS.div(resultsMS.sum(axis=1), axis=0)
    resultsMS_Norm[subject].reset_index(inplace=True)
    
    print(resultsMS_Norm[subject].head())
    
    # Dataframe with average
    
del resultsDF, resultsMS

   Year  Level 1 Math  Level 2 Math  Level 3 Math  Level 4 Math
0  2013         0.384         0.351         0.164         0.101
1  2014         0.373         0.338         0.173         0.116
2  2015         0.364         0.331         0.172         0.133
3  2016         0.351         0.325         0.166         0.157
4  2017         0.367         0.307         0.173         0.153
   Year  Level 1 ELA  Level 2 ELA  Level 3 ELA  Level 4 ELA
0  2013        0.384        0.351        0.164        0.101
1  2014        0.373        0.338        0.173        0.116
2  2015        0.364        0.331        0.172        0.133
3  2016        0.351        0.325        0.166        0.157
4  2017        0.367        0.307        0.173        0.153


In [ ]:
# Get 12 years average test result

resultsMS_12y_AVG = {}

for subject in subjects:
        
    resultsDF = cityResultsDFs[subject]
    
    # Dataframe with only grades 6-8 results (middle schools and K-8) by years
    resultsMS = resultsDF[(resultsDF['Grade'] >= 6)&(resultsDF['Grade'] <= 8)]
    
    # Dataframe with all results summed by all years
    columns_to_sum = ['# Level 1','# Level 2','# Level 3','# Level 4']
    resultsMS = resultsMS[columns_to_sum]
    
    # Change column names to include subject
    resultsMS.columns = [f'Level 1 {subject}',f'Level 2 {subject}',f'Level 3 {subject}',f'Level 4 {subject}']
    resultsMS = resultsMS.sum() #Dataframe got converted into a series, needs fixing later

    # Change column names to include subject
    resultsMS.columns = [f'Level 1 {subject}',f'Level 2 {subject}',f'Level 3 {subject}',f'Level 4 {subject}']
    
    # Dataframe for middle schools grades with normalized values
    resultsMS_12y_AVG[subject] = resultsMS.div(resultsMS.sum(axis=0))
    
    # Convert the series back into a dataframe
    resultsMS_12y_AVG[subject] = resultsMS_12y_AVG[subject].to_frame().T # Transpose to flip rows and columns
   
    print(resultsMS_12y_AVG[subject].head())
        
del resultsDF, resultsMS

   Level 1 Math  Level 2 Math  Level 3 Math  Level 4 Math
0         0.346         0.288         0.200         0.166
   Level 1 ELA  Level 2 ELA  Level 3 ELA  Level 4 ELA
0        0.296        0.295        0.220        0.189


In [27]:
# Make a merged city dataframe with both math and ELA results

DFs = list(resultsMS_12y_AVG.values())
cityAVG12yDF = pd.merge(DFs[0], DFs[1], left_index=True, right_index=True)
print(cityAVG12yDF.head())

del DFs

   Level 1 Math  Level 2 Math  Level 3 Math  Level 4 Math  Level 1 ELA  \
0         0.346         0.288         0.200         0.166        0.296   

   Level 2 ELA  Level 3 ELA  Level 4 ELA  
0        0.295        0.220        0.189  


In [28]:
# Adding column with sum of shares of test results of level 4 in math and ELA

cityAVG12yDF['Level 4 Math+Ela'] = cityAVG12yDF['Level 4 Math']+cityAVG12yDF['Level 4 ELA']

In [29]:
# Add column 'Year' to 'cityAVG12yDF' dataframe to be able to merge the dataframes later

cityAVG12yDF.insert(0, 'Year',0)

In [30]:
cityAVG12yDF.head()

,Year,Level 1 Math,Level 2 Math,Level 3 Math,Level 4 Math,Level 1 ELA,Level 2 ELA,Level 3 ELA,Level 4 ELA,Level 4 Math+Ela
0,0,0.346,0.288,0.200,0.166,0.296,0.295,0.220,0.189,0.355


In [31]:
# Make a merged city dataframe with both math and ELA results by years

DFs = list(resultsMS_Norm.values())
cityResultsDF = pd.merge(DFs[0], DFs[1], on = ['Year'], how = 'inner')
print(cityResultsDF.head(15))

del DFs

    Year  Level 1 Math  Level 2 Math  Level 3 Math  Level 4 Math  Level 1 ELA  \
0   2013         0.384         0.351         0.164         0.101        0.384   
1   2014         0.373         0.338         0.173         0.116        0.373   
2   2015         0.364         0.331         0.172         0.133        0.364   
3   2016         0.351         0.325         0.166         0.157        0.351   
4   2017         0.367         0.307         0.173         0.153        0.367   
5   2018         0.360         0.260         0.189         0.191        0.236   
6   2019         0.341         0.248         0.195         0.216        0.253   
7   2022         0.424         0.249         0.159         0.167        0.201   
8   2023         0.295         0.236         0.281         0.189        0.215   
9   2024         0.252         0.234         0.295         0.219        0.241   
10  2025         0.241         0.229         0.298         0.232        0.212   

    Level 2 ELA  Level 3 EL

In [32]:
# Calculating the column with sums of shares of level 4 results

cityResultsDF['Level 4 Math+Ela'] = cityResultsDF['Level 4 Math']+cityResultsDF['Level 4 ELA']
cityResultsDF.head(13)

,Year,Level 1 Math,Level 2 Math,Level 3 Math,Level 4 Math,Level 1 ELA,Level 2 ELA,Level 3 ELA,Level 4 ELA,Level 4 Math+Ela
0,2013,0.384,0.351,0.164,0.101,0.384,0.351,0.164,0.101,0.202
1,2014,0.373,0.338,0.173,0.116,0.373,0.338,0.173,0.116,0.233
2,2015,0.364,0.331,0.172,0.133,0.364,0.331,0.172,0.133,0.266
3,2016,0.351,0.325,0.166,0.157,0.351,0.325,0.166,0.157,0.315
4,2017,0.367,0.307,0.173,0.153,0.367,0.307,0.173,0.153,0.306
5,2018,0.360,0.260,0.189,0.191,0.236,0.289,0.249,0.225,0.416
6,2019,0.341,0.248,0.195,0.216,0.253,0.275,0.233,0.240,0.456
7,2022,0.424,0.249,0.159,0.167,0.201,0.259,0.260,0.280,0.447
8,2023,0.295,0.236,0.281,0.189,0.215,0.253,0.298,0.235,0.423
9,2024,0.252,0.234,0.295,0.219,0.241,0.244,0.289,0.226,0.445


In [ ]:
# Comparison = '2025 - 10 year average' to see citywide trend 

TenyAVG_2025DF = (cityResultsDF.iloc[10] - cityAVG12yDF.iloc[0])
TenyAVG_2025DF = TenyAVG_2025DF.drop('Year')
TenyAVG_2025DF

Level 1 Math       -0.105
Level 2 Math       -0.060
Level 3 Math        0.098
Level 4 Math        0.066
Level 1 ELA        -0.083
Level 2 ELA        -0.053
Level 3 ELA         0.066
Level 4 ELA         0.070
Level 4 Math+Ela    0.136
dtype: float64

<a id="middle"></a> 
### Getting the test results for middle schools and calculate comparison indicator by school

#### Prepare schools dataframe with only middle school tests results (grades 6-8)

In [34]:
# Select middle school grades results from the dataframes with math and ELA tests results by schools

resultsMS_bySchl_Norm ={}

for subject in subjects:
    
    
    resultsDF = resultsDFs[subject]
    
    # Dataframe with only grades 6-8 results (middle schools and K-8) by years
    resultsMS_bySchl = resultsDF[(resultsDF['Grade'] >= 6)&(resultsDF['Grade'] <= 8)]
    resultsMS_bySchl = resultsMS_bySchl.groupby(['DBN', 'School Name', 'Year'])[['# Level 1','# Level 2','# Level 3','# Level 4']].sum()
    
    # Change column names to include subject
    resultsMS_bySchl.columns = [f'Level 1 {subject}',f'Level 2 {subject}',f'Level 3 {subject}',f'Level 4 {subject}']
    
    # Dataframe for middle schools by years with normalized values
    resultsMS_bySchl_Norm[subject] = resultsMS_bySchl.div(resultsMS_bySchl.sum(axis=1), axis=0)
    resultsMS_bySchl_Norm[subject].reset_index(inplace=True)
    
    print(resultsMS_bySchl_Norm[subject].head(13))
    
del resultsDF, resultsMS_bySchl

       DBN                     School Name  Year  Level 1 Math  Level 2 Math  \
0   01M015       P.S. 015 ROBERTO CLEMENTE  2023           NaN           NaN   
1   01M034  P.S. 034 FRANKLIN D. ROOSEVELT  2013         0.302         0.416   
2   01M034  P.S. 034 FRANKLIN D. ROOSEVELT  2014         0.336         0.375   
3   01M034  P.S. 034 FRANKLIN D. ROOSEVELT  2015         0.361         0.392   
4   01M034  P.S. 034 FRANKLIN D. ROOSEVELT  2016         0.420         0.408   
5   01M034  P.S. 034 FRANKLIN D. ROOSEVELT  2017         0.412         0.420   
6   01M034  P.S. 034 FRANKLIN D. ROOSEVELT  2018         0.580         0.277   
7   01M034  P.S. 034 FRANKLIN D. ROOSEVELT  2019         0.524         0.311   
8   01M034  P.S. 034 FRANKLIN D. ROOSEVELT  2022         0.738         0.154   
9   01M034  P.S. 034 FRANKLIN D. ROOSEVELT  2023         0.366         0.366   
10  01M034  P.S. 034 FRANKLIN D. ROOSEVELT  2024         0.279         0.395   
11  01M034  P.S. 034 FRANKLIN D. ROOSEVE

In [35]:
# Make a merged dataframe with both math and ELA results

DFs = list(resultsMS_bySchl_Norm.values())
allResultsDF = pd.merge(DFs[0], DFs[1], on = ['DBN', 'Year'], how = 'inner', suffixes=('', '_drop'))
allResultsDF = allResultsDF.loc[:, ~allResultsDF.columns.str.endswith('_drop')]
allResultsDF.head(5)

del DFs

In [36]:
# Add colomn with sum of shares of level4 students by math and level4 students ELA

allResultsDF['Level 4 Math+Ela'] = allResultsDF[f'Level 4 {subjects[0]}']+allResultsDF[f'Level 4 {subjects[1]}']
allResultsDF.head(10)

,DBN,School Name,Year,Level 1 Math,Level 2 Math,Level 3 Math,Level 4 Math,Level 1 ELA,Level 2 ELA,Level 3 ELA,Level 4 ELA,Level 4 Math+Ela
0,01M015,P.S. 015 ROBERTO CLEMENTE,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01M034,P.S. 034 FRANKLIN D. ROOSEVELT,2013,0.302,0.416,0.195,0.087,0.366,0.524,0.097,0.014,0.101
2,01M034,P.S. 034 FRANKLIN D. ROOSEVELT,2014,0.336,0.375,0.230,0.059,0.301,0.477,0.176,0.046,0.105
3,01M034,P.S. 034 FRANKLIN D. ROOSEVELT,2015,0.361,0.392,0.190,0.057,0.250,0.461,0.250,0.039,0.096
4,01M034,P.S. 034 FRANKLIN D. ROOSEVELT,2016,0.420,0.408,0.127,0.045,0.237,0.481,0.231,0.051,0.096
5,01M034,P.S. 034 FRANKLIN D. ROOSEVELT,2017,0.412,0.420,0.137,0.031,0.187,0.511,0.245,0.058,0.088
6,01M034,P.S. 034 FRANKLIN D. ROOSEVELT,2018,0.580,0.277,0.092,0.050,0.284,0.414,0.172,0.129,0.180
7,01M034,P.S. 034 FRANKLIN D. ROOSEVELT,2019,0.524,0.311,0.146,0.019,0.393,0.402,0.121,0.084,0.104
8,01M034,P.S. 034 FRANKLIN D. ROOSEVELT,2022,0.738,0.154,0.092,0.015,0.337,0.421,0.147,0.095,0.110
9,01M034,P.S. 034 FRANKLIN D. ROOSEVELT,2023,0.366,0.366,0.220,0.049,0.241,0.430,0.241,0.089,0.137


<a id="best"></a> 
#### Select schools with the best results for all middle school grades in 2025
Optional step,except for the first cell (dataframe for 2025), is not needed for the rest of the analysis.

In [37]:
# This dataframe for 2025 is used later to compare school progress to the citywide progress

allSchools2025 = allResultsDF[(allResultsDF['Year'] == 2025)]
allSchools2025.head()

,DBN,School Name,Year,Level 1 Math,Level 2 Math,Level 3 Math,Level 4 Math,Level 1 ELA,Level 2 ELA,Level 3 ELA,Level 4 ELA,Level 4 Math+Ela
11,01M034,P.S. 034 FRANKLIN D. ROOSEVELT,2025,0.333,0.487,0.154,0.026,0.339,0.306,0.274,0.081,0.106
22,01M140,P.S. 140 NATHAN STRAUS,2025,0.369,0.298,0.270,0.064,0.441,0.280,0.212,0.068,0.132
33,01M184,P.S. 184M SHUANG WEN,2025,0.048,0.042,0.259,0.651,0.091,0.086,0.284,0.539,1.190
44,01M188,P.S. 188 THE ISLAND SCHOOL,2025,0.095,0.060,0.452,0.393,0.254,0.289,0.317,0.141,0.534
55,01M332,UNIVERSITY NEIGHBORHOOD MIDDLE SCHOOL,2025,0.422,0.349,0.145,0.084,0.250,0.304,0.304,0.143,0.227


Optional steps if desired:

<a id="ten"></a> 
#### Create dataframe with average 2013-2025 math and ELA test results for all middle school grades

In [38]:
# Make a merged dataframe with both Math and ELA average 2013-2025 results by schools

resultsMS_top50_AVG2013_25 = {}
resultsMS_AVG2013_25 = {}

for subject in subjects:
    
    resultsDF = resultsDFs[subject]
    
    # Dataframe with only grades 6-8 results (middle schools and K-8) by schools
    resultsMS_bySchl_sumed = resultsDF[(resultsDF['Grade'] >= 6)&(resultsDF['Grade'] <= 8)].groupby(['DBN', 'School Name'])[['# Level 1','# Level 2','# Level 3','# Level 4']].sum()
    # Rename columns
    resultsMS_bySchl_sumed.columns = [f'# Level 1 {subject}',f'# Level 2 {subject}',f'# Level 3 {subject}',f'# Level 4 {subject}']

    
    # Dataframe for middle schools by years with normalized values
    resultsMS_bySchl_sumed_Norm = resultsMS_bySchl_sumed.div(resultsMS_bySchl_sumed.sum(axis=1), axis=0)
    resultsMS_bySchl_sumed_Norm.columns = [f'12yrs avg Lvl 1 {subject}',f'12yrs avg Lvl 2 {subject}',f'12yrs avg Lvl 3 {subject}',f'12yrs avg Lvl 4 {subject}']
    resultsMS_bySchl_sumed_Norm.reset_index(inplace = True)
    
    # Add the dataframe to the respective dictionnary 
    resultsMS_AVG2013_25[subject] = resultsMS_bySchl_sumed_Norm
    print(len(resultsMS_AVG2013_25[subject]))
    
del resultsDF, resultsMS_bySchl_sumed_Norm, resultsMS_bySchl_sumed

541
544


In [39]:
# Make a merged dataframe with both Math and ELA average 2013-2025 results by schools

AVG2013_25_DFs = list(resultsMS_AVG2013_25.values())
allResultsAVG2013_25DF = pd.merge(AVG2013_25_DFs[0], AVG2013_25_DFs[1], on = ['DBN','School Name'], how = 'inner', suffixes=('', '_drop'))
allResultsAVG2013_25DF = allResultsAVG2013_25DF.loc[:, ~allResultsAVG2013_25DF.columns.str.endswith('_drop')]
allResultsAVG2013_25DF['12yrs avg Lvl 4 Math+Ela'] = allResultsAVG2013_25DF[f'12yrs avg Lvl 4 {subjects[0]}']+allResultsAVG2013_25DF[f'12yrs avg Lvl 4 {subjects[1]}']

del AVG2013_25_DFs

In [40]:
allResultsAVG2013_25DF.head()

,DBN,School Name,12yrs avg Lvl 1 Math,12yrs avg Lvl 2 Math,12yrs avg Lvl 3 Math,12yrs avg Lvl 4 Math,12yrs avg Lvl 1 ELA,12yrs avg Lvl 2 ELA,12yrs avg Lvl 3 ELA,12yrs avg Lvl 4 ELA,12yrs avg Lvl 4 Math+Ela
0,01M015,P.S. 015 ROBERTO CLEMENTE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01M034,P.S. 034 FRANKLIN D. ROOSEVELT,0.411,0.373,0.169,0.048,0.283,0.455,0.199,0.064,0.111
2,01M140,P.S. 140 NATHAN STRAUS,0.458,0.324,0.167,0.050,0.343,0.407,0.189,0.061,0.111
3,01M184,P.S. 184M SHUANG WEN,0.052,0.102,0.245,0.600,0.062,0.160,0.329,0.449,1.049
4,01M188,P.S. 188 THE ISLAND SCHOOL,0.272,0.366,0.244,0.118,0.325,0.421,0.203,0.052,0.169


In [41]:
# Merging in the 2025 results

allResultsAVG2013_25DF = allResultsAVG2013_25DF.merge(allSchools2025, left_on = 'School Name', right_on = 'School Name',  suffixes=('', '_drop'))
allResultsAVG2013_25DF = allResultsAVG2013_25DF.loc[:, ~allResultsAVG2013_25DF.columns.str.endswith('_drop')]
allResultsAVG2013_25DF.head()

,DBN,School Name,12yrs avg Lvl 1 Math,12yrs avg Lvl 2 Math,12yrs avg Lvl 3 Math,12yrs avg Lvl 4 Math,12yrs avg Lvl 1 ELA,12yrs avg Lvl 2 ELA,12yrs avg Lvl 3 ELA,12yrs avg Lvl 4 ELA,...,Year,Level 1 Math,Level 2 Math,Level 3 Math,Level 4 Math,Level 1 ELA,Level 2 ELA,Level 3 ELA,Level 4 ELA,Level 4 Math+Ela
0,01M034,P.S. 034 FRANKLIN D. ROOSEVELT,0.411,0.373,0.169,0.048,0.283,0.455,0.199,0.064,...,2025,0.333,0.487,0.154,0.026,0.339,0.306,0.274,0.081,0.106
1,01M140,P.S. 140 NATHAN STRAUS,0.458,0.324,0.167,0.050,0.343,0.407,0.189,0.061,...,2025,0.369,0.298,0.270,0.064,0.441,0.280,0.212,0.068,0.132
2,01M184,P.S. 184M SHUANG WEN,0.052,0.102,0.245,0.600,0.062,0.160,0.329,0.449,...,2025,0.048,0.042,0.259,0.651,0.091,0.086,0.284,0.539,1.190
3,01M188,P.S. 188 THE ISLAND SCHOOL,0.272,0.366,0.244,0.118,0.325,0.421,0.203,0.052,...,2025,0.095,0.060,0.452,0.393,0.254,0.289,0.317,0.141,0.534
4,01M332,UNIVERSITY NEIGHBORHOOD MIDDLE SCHOOL,0.599,0.250,0.104,0.048,0.361,0.340,0.212,0.087,...,2025,0.422,0.349,0.145,0.084,0.250,0.304,0.304,0.143,0.227


In [42]:
# Adding comparison between results of 2025 and 2013-2023 average

allResultsAVG2013_25DF['2025-12yAVG'] = allResultsAVG2013_25DF['Level 4 Math+Ela'] - allResultsAVG2013_25DF['12yrs avg Lvl 4 Math+Ela']
allResultsAVG2013_25DF.head()

,DBN,School Name,12yrs avg Lvl 1 Math,12yrs avg Lvl 2 Math,12yrs avg Lvl 3 Math,12yrs avg Lvl 4 Math,12yrs avg Lvl 1 ELA,12yrs avg Lvl 2 ELA,12yrs avg Lvl 3 ELA,12yrs avg Lvl 4 ELA,...,Level 1 Math,Level 2 Math,Level 3 Math,Level 4 Math,Level 1 ELA,Level 2 ELA,Level 3 ELA,Level 4 ELA,Level 4 Math+Ela,2025-12yAVG
0,01M034,P.S. 034 FRANKLIN D. ROOSEVELT,0.411,0.373,0.169,0.048,0.283,0.455,0.199,0.064,...,0.333,0.487,0.154,0.026,0.339,0.306,0.274,0.081,0.106,-0.005
1,01M140,P.S. 140 NATHAN STRAUS,0.458,0.324,0.167,0.050,0.343,0.407,0.189,0.061,...,0.369,0.298,0.270,0.064,0.441,0.280,0.212,0.068,0.132,0.021
2,01M184,P.S. 184M SHUANG WEN,0.052,0.102,0.245,0.600,0.062,0.160,0.329,0.449,...,0.048,0.042,0.259,0.651,0.091,0.086,0.284,0.539,1.190,0.140
3,01M188,P.S. 188 THE ISLAND SCHOOL,0.272,0.366,0.244,0.118,0.325,0.421,0.203,0.052,...,0.095,0.060,0.452,0.393,0.254,0.289,0.317,0.141,0.534,0.364
4,01M332,UNIVERSITY NEIGHBORHOOD MIDDLE SCHOOL,0.599,0.250,0.104,0.048,0.361,0.340,0.212,0.087,...,0.422,0.349,0.145,0.084,0.250,0.304,0.304,0.143,0.227,0.093


In [43]:
allResultsAVG2013_25DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481 entries, 0 to 480
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   DBN                       481 non-null    object 
 1   School Name               481 non-null    object 
 2   12yrs avg Lvl 1 Math      480 non-null    float64
 3   12yrs avg Lvl 2 Math      480 non-null    float64
 4   12yrs avg Lvl 3 Math      480 non-null    float64
 5   12yrs avg Lvl 4 Math      480 non-null    float64
 6   12yrs avg Lvl 1 ELA       480 non-null    float64
 7   12yrs avg Lvl 2 ELA       480 non-null    float64
 8   12yrs avg Lvl 3 ELA       480 non-null    float64
 9   12yrs avg Lvl 4 ELA       480 non-null    float64
 10  12yrs avg Lvl 4 Math+Ela  480 non-null    float64
 11  Year                      481 non-null    int64  
 12  Level 1 Math              480 non-null    float64
 13  Level 2 Math              480 non-null    float64
 14  Level 3 Ma

<a id="three"></a> 
#### Create dataframe with average 2019-2025 (last 4 tests) math and ela test results for all middle school grades

In [44]:
# Make a merged dataframe with both math and ELA average 2019-2025 results 

resultsMS_AVG2019_25 = {}

for subject in subjects:
    
    resultsDF = resultsDFs[subject]
    
    # Dataframe with only grades 6-8 results (middle schools and K-8) by schools
    resultsMS_bySchl_sumed = resultsDF[((resultsDF['Grade'] >= 6)&(resultsDF['Grade'] <= 8))&(resultsDF['Year'] >= 2019)].groupby(['DBN', 'School Name'])[['# Level 1','# Level 2','# Level 3','# Level 4']].sum()
    # Rename columns
    resultsMS_bySchl_sumed.columns = [f'# Level 1 {subject}',f'# Level 2 {subject}',f'# Level 3 {subject}',f'# Level 4 {subject}']

    
    # Dataframe for middle schools by years with normalized values
    resultsMS_bySchl_sumed_Norm = resultsMS_bySchl_sumed.div(resultsMS_bySchl_sumed.sum(axis=1), axis=0)
    resultsMS_bySchl_sumed_Norm.columns = [f'5yrs avg Lvl 1 {subject}',f'5yrs avg Lvl 2 {subject}',f'5yrs avg Lvl 3 {subject}',f'5yrs avg Lvl 4 {subject}']
    resultsMS_bySchl_sumed_Norm.reset_index(inplace = True)
    
    # Add the dataframe to the respective dictionnary     
    resultsMS_AVG2019_25[subject] = resultsMS_bySchl_sumed_Norm
    print(len(resultsMS_AVG2019_25[subject]))
    
del resultsDF, resultsMS_bySchl_sumed_Norm, resultsMS_bySchl_sumed

511
514


In [45]:
# Make a merged dataframe with both Math and ELA average 2019-2023 results 

AVG2019_25_DFs = list(resultsMS_AVG2019_25.values())
allResultsAVG2019_25DF = pd.merge(AVG2019_25_DFs[0], AVG2019_25_DFs[1], on = ['DBN','School Name'], how = 'inner')
allResultsAVG2019_25DF['5yrs avg Lvl 4 Math+Ela'] = allResultsAVG2019_25DF[f'5yrs avg Lvl 4 {subjects[0]}']+allResultsAVG2019_25DF[f'5yrs avg Lvl 4 {subjects[1]}']

del AVG2019_25_DFs

In [46]:
allResultsAVG2019_25DF.head()

,DBN,School Name,5yrs avg Lvl 1 Math,5yrs avg Lvl 2 Math,5yrs avg Lvl 3 Math,5yrs avg Lvl 4 Math,5yrs avg Lvl 1 ELA,5yrs avg Lvl 2 ELA,5yrs avg Lvl 3 ELA,5yrs avg Lvl 4 ELA,5yrs avg Lvl 4 Math+Ela
0,01M015,P.S. 015 ROBERTO CLEMENTE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01M034,P.S. 034 FRANKLIN D. ROOSEVELT,0.448,0.346,0.177,0.029,0.310,0.399,0.204,0.087,0.116
2,01M140,P.S. 140 NATHAN STRAUS,0.439,0.283,0.210,0.069,0.341,0.323,0.238,0.098,0.166
3,01M184,P.S. 184M SHUANG WEN,0.046,0.075,0.237,0.642,0.066,0.114,0.290,0.530,1.172
4,01M188,P.S. 188 THE ISLAND SCHOOL,0.152,0.257,0.404,0.187,0.229,0.396,0.292,0.083,0.270


<a id="final"></a> 
### Create final dataframe with data for mapping

In [47]:
# Merge dataframes with average 10 years and last 4 tests results

schoolsAllData = pd.merge(allResultsAVG2013_25DF, allResultsAVG2019_25DF, left_on = ['DBN', 'School Name'], right_on = ['DBN', 'School Name'], how = 'inner')
schoolsAllData.head()

,DBN,School Name,12yrs avg Lvl 1 Math,12yrs avg Lvl 2 Math,12yrs avg Lvl 3 Math,12yrs avg Lvl 4 Math,12yrs avg Lvl 1 ELA,12yrs avg Lvl 2 ELA,12yrs avg Lvl 3 ELA,12yrs avg Lvl 4 ELA,...,2025-12yAVG,5yrs avg Lvl 1 Math,5yrs avg Lvl 2 Math,5yrs avg Lvl 3 Math,5yrs avg Lvl 4 Math,5yrs avg Lvl 1 ELA,5yrs avg Lvl 2 ELA,5yrs avg Lvl 3 ELA,5yrs avg Lvl 4 ELA,5yrs avg Lvl 4 Math+Ela
0,01M034,P.S. 034 FRANKLIN D. ROOSEVELT,0.411,0.373,0.169,0.048,0.283,0.455,0.199,0.064,...,-0.005,0.448,0.346,0.177,0.029,0.310,0.399,0.204,0.087,0.116
1,01M140,P.S. 140 NATHAN STRAUS,0.458,0.324,0.167,0.050,0.343,0.407,0.189,0.061,...,0.021,0.439,0.283,0.210,0.069,0.341,0.323,0.238,0.098,0.166
2,01M184,P.S. 184M SHUANG WEN,0.052,0.102,0.245,0.600,0.062,0.160,0.329,0.449,...,0.140,0.046,0.075,0.237,0.642,0.066,0.114,0.290,0.530,1.172
3,01M188,P.S. 188 THE ISLAND SCHOOL,0.272,0.366,0.244,0.118,0.325,0.421,0.203,0.052,...,0.364,0.152,0.257,0.404,0.187,0.229,0.396,0.292,0.083,0.270
4,01M332,UNIVERSITY NEIGHBORHOOD MIDDLE SCHOOL,0.599,0.250,0.104,0.048,0.361,0.340,0.212,0.087,...,0.093,0.539,0.255,0.144,0.062,0.302,0.300,0.283,0.114,0.176


In [48]:
# If needed, the dataframe can be saved to csv for safekeeping or for reuse without repeating 
# the steps above

filename = 'schools2013_2025_AVG.csv'
name = os.path.join(basePath, outputFolder,filename)
schoolsAllData.to_csv(name, index = True)
del filename, name

In [49]:
schoolsAllData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481 entries, 0 to 480
Data columns (total 31 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   DBN                       481 non-null    object 
 1   School Name               481 non-null    object 
 2   12yrs avg Lvl 1 Math      480 non-null    float64
 3   12yrs avg Lvl 2 Math      480 non-null    float64
 4   12yrs avg Lvl 3 Math      480 non-null    float64
 5   12yrs avg Lvl 4 Math      480 non-null    float64
 6   12yrs avg Lvl 1 ELA       480 non-null    float64
 7   12yrs avg Lvl 2 ELA       480 non-null    float64
 8   12yrs avg Lvl 3 ELA       480 non-null    float64
 9   12yrs avg Lvl 4 ELA       480 non-null    float64
 10  12yrs avg Lvl 4 Math+Ela  480 non-null    float64
 11  Year                      481 non-null    int64  
 12  Level 1 Math              480 non-null    float64
 13  Level 2 Math              480 non-null    float64
 14  Level 3 Ma

<a id="status"></a> 
#### Adding school status (citywide, boroughwide) and the diversity data to the dataframe with all tests resuls

In [50]:
# Preparing the demographic data

demogData.columns = [col.replace('/', '_') for col in demogData.columns]

In [51]:
# Selecting the columns needed for analysis from demography data

cols = ['DBN', 'Year', 'Total Enrollment', '% Asian', '% Black', '% Hispanic', '% Multi-Racial', '% Native American', '% White', '% Missing Race_Ethnicity Data']
diversityData = demogData[cols]
index = diversityData['Year'] == '2022-23'
diversityData = diversityData[index]

In [52]:
len(diversityData)

1890

In [53]:
# Merging the school diversity data and school status (open to city/borough) data

diversityStatusData = pd.merge(diversityData, statusData, on = 'DBN', how = 'outer')
len(diversityStatusData)

1890

In [54]:
# Merging schools data (short version) for analysis with demographic and status data

schoolsMergedData = schoolsAllData.merge(diversityStatusData, on = 'DBN', how = 'inner', suffixes=('', '_drop'))
schoolsMergedData = schoolsMergedData.loc[:, ~schoolsMergedData.columns.str.endswith('_drop')]

In [55]:
len(schoolsMergedData)

475

In [56]:
schoolsMergedData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 475 entries, 0 to 474
Data columns (total 41 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   DBN                            475 non-null    object 
 1   School Name                    475 non-null    object 
 2   12yrs avg Lvl 1 Math           474 non-null    float64
 3   12yrs avg Lvl 2 Math           474 non-null    float64
 4   12yrs avg Lvl 3 Math           474 non-null    float64
 5   12yrs avg Lvl 4 Math           474 non-null    float64
 6   12yrs avg Lvl 1 ELA            474 non-null    float64
 7   12yrs avg Lvl 2 ELA            474 non-null    float64
 8   12yrs avg Lvl 3 ELA            474 non-null    float64
 9   12yrs avg Lvl 4 ELA            474 non-null    float64
 10  12yrs avg Lvl 4 Math+Ela       474 non-null    float64
 11  Year                           475 non-null    int64  
 12  Level 1 Math                   474 non-null    flo

#### Read schools geolocation file

In [57]:
## Read GeoJSON into data frame
SchoolsFile = 'NYC_K-12_schools_public.geojson'
NYCSchoolsPath = os.path.join(basePath, dataFolder, SchoolsFile)
NYCSchoolsData = gpd.read_file(NYCSchoolsPath)

DistrictsFile = 'School Districts.geojson'
NYCDistrictsPath = os.path.join(basePath, dataFolder, DistrictsFile)
NYCDistrictsData = gpd.read_file(NYCDistrictsPath)

<a id="match"></a> 
#### Matching the schools names from GeoJSON schools location file and the results dataframe and merging

In [58]:
#NYCSchoolsData.info() #Too many columns --> make a smaller copy
NYCSchoolsDataShort = NYCSchoolsData[['OBJECTID', 'LEGAL_NAME', 'PHYSADDRLINE1', 'PHYSCITY', 'COUNTY_DESC', 'RECORD_TYPE_DESC', 'SDL_DESC', 'geometry']]
NYCSchoolsDataShort.head()

,OBJECTID,LEGAL_NAME,PHYSADDRLINE1,PHYSCITY,COUNTY_DESC,RECORD_TYPE_DESC,SDL_DESC,geometry
0,54,PS 11 THOMAS DONGAN SCHOOL,85 GARRETSON AVE,STATEN ISLAND,RICHMOND,PUBLIC SCHOOL (IMF),NYC GEOG DIST 31,POINT (576322.056 4493696.89)
1,55,SCHOOL WITHOUT WALLS,207 TRINITY PL,NEW YORK,NEW YORK,PUBLIC SCHOOL (IMF),NYC GEOG DIST 2,POINT (583425.686 4506914.562)
2,131,YOUNG WOMEN'S LEADERSHIP SCHOOL,140 W 140TH ST,NEW YORK,NEW YORK,PUBLIC SCHOOL (IMF),NYC GEOG DIST 4,POINT (589347.579 4519091.922)
3,132,EDWARD A REYNOLDS WEST SIDE HIGH SCHOOL,105 E 106TH ST,NEW YORK,NEW YORK,PUBLIC SCHOOL (IMF),NYC GEOG DIST 3,POINT (588814.806 4516292.893)
4,134,ASPIRATIONS DIPLOMA PLUS HIGH SCHOOL,1150 E NEW YORK AVE,BROOKLYN,KINGS,PUBLIC SCHOOL (IMF),NYC GEOG DIST 17,POINT (590940.002 4502277.994)


In [59]:
# Matching the school all data file with spatial data (geojson of schools locations)

tqdm.pandas(desc="Matching Names")

# Matching names from resultsMS_bySchl_Norm[subject] to NYCSchoolsDataShort
matched_tuples = schoolsMergedData['School Name'].progress_apply(lambda x: match_name(x, NYCSchoolsDataShort['LEGAL_NAME'], min_score=80))

print('Done.')

Matching Names: 100%|██████████| 475/475 [01:43<00:00,  4.61it/s]

Done.


In [60]:
print('Appending mathes to the dataframe.')
schoolsMergedData['matched_name'] = list(zip(*matched_tuples))[0]
schoolsMergedData['matched_score'] = list(zip(*matched_tuples))[1]
print('Done.')

Appending mathes to the dataframe.
Done.


In [61]:
schoolsMergedData.head()

,DBN,School Name,12yrs avg Lvl 1 Math,12yrs avg Lvl 2 Math,12yrs avg Lvl 3 Math,12yrs avg Lvl 4 Math,12yrs avg Lvl 1 ELA,12yrs avg Lvl 2 ELA,12yrs avg Lvl 3 ELA,12yrs avg Lvl 4 ELA,...,% Black,% Hispanic,% Multi-Racial,% Native American,% White,% Missing Race_Ethnicity Data,Open to,Comments,matched_name,matched_score
0,01M034,P.S. 034 FRANKLIN D. ROOSEVELT,0.411,0.373,0.169,0.048,0.283,0.455,0.199,0.064,...,0.418,0.527,0.000,0.009,0.027,0.000,NaN,NaN,PS 34 FRANKLIN D ROOSEVELT,93
1,01M140,P.S. 140 NATHAN STRAUS,0.458,0.324,0.167,0.050,0.343,0.407,0.189,0.061,...,0.206,0.728,0.007,0.000,0.030,0.007,NaN,NaN,PS 140 NATHAN STRAUS,98
2,01M184,P.S. 184M SHUANG WEN,0.052,0.102,0.245,0.600,0.062,0.160,0.329,0.449,...,0.026,0.123,0.089,0.003,0.078,0.003,NaN,NaN,PS 184 SHUANG WEN,94
3,01M188,P.S. 188 THE ISLAND SCHOOL,0.272,0.366,0.244,0.118,0.325,0.421,0.203,0.052,...,0.331,0.573,0.015,0.012,0.017,0.003,NaN,NaN,PS 188 ISLAND SCHOOL (THE),98
4,01M332,UNIVERSITY NEIGHBORHOOD MIDDLE SCHOOL,0.599,0.250,0.104,0.048,0.361,0.340,0.212,0.087,...,0.256,0.600,0.019,0.013,0.069,0.013,NaN,NaN,UNIVERSITY NEIGHBORHOOD MIDDLE SCHOOL,100


In [62]:
# Merging DataFrames based on the matched name
schoolsAllData_mappable = pd.merge(NYCSchoolsDataShort,schoolsMergedData, left_on='LEGAL_NAME', right_on='matched_name')

In [63]:
schoolsAllData_mappable.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 455 entries, 0 to 454
Data columns (total 51 columns):
 #   Column                         Non-Null Count  Dtype   
---  ------                         --------------  -----   
 0   OBJECTID                       455 non-null    int32   
 1   LEGAL_NAME                     455 non-null    object  
 2   PHYSADDRLINE1                  455 non-null    object  
 3   PHYSCITY                       455 non-null    object  
 4   COUNTY_DESC                    455 non-null    object  
 5   RECORD_TYPE_DESC               455 non-null    object  
 6   SDL_DESC                       455 non-null    object  
 7   geometry                       455 non-null    geometry
 8   DBN                            455 non-null    object  
 9   School Name                    455 non-null    object  
 10  12yrs avg Lvl 1 Math           454 non-null    float64 
 11  12yrs avg Lvl 2 Math           454 non-null    float64 
 12  12yrs avg Lvl 3 Math        

In [64]:
print(schoolsAllData_mappable['matched_name'].isnull().sum())

0


<a id="plots"></a> 
#### Adding history ELA/math results, diversity data as plots to the geodata frame and saving into GeoJSON file

In [66]:
# Make piecharts for popups in the map and add them as columns to the mappable dataframe

# Initialize AVGDF_mappable_plots with the original DataFrame to preserve its content across merges
schools_mappable_plots = schoolsAllData_mappable.copy()

# Set interactive mode off
plt.ioff()

# list of schools names

schoolsNames = schoolsAllData_mappable['DBN'].to_list()

# Create disctionnary to hold the dataframes by schools
schoolDFs = {}

# Make dataframes by schools 
for name in schoolsNames:
    dfName = name
    schoolDFs[dfName] = schools_mappable_plots[schools_mappable_plots['DBN'] == name]

plots = []
plotsDFs = {}

print("Making test results plots ...")

columns_to_plot = ['% Asian', '% Black', '% Hispanic', '% Multi-Racial', '% Native American', '% White', '% Missing Race_Ethnicity Data']  
# Plot dataframes by school
for schoolDF, current_dataframe in tqdm(schoolDFs.items()):
    # schoolDF contains the name of the dataframe
    # current_dataframe contains the dataframe itself

        # Do something with current_dataframe
        # Create a plot
        fig = create_chart(current_dataframe, schoolDF, columns_to_plot)

        # Convert the plot to a PNG image and then encode it
        io_buf = BytesIO()
        fig.savefig(io_buf, format='png', bbox_inches='tight')
        # Close the figure
        plt.close()        
        #Reading file to get the base64 string
        io_buf.seek(0)
        base64_string = base64.b64encode(io_buf.read()).decode('utf8')

        pair = (schoolDF, base64_string)

        plots.append(pair)

print('Adding plots to the data frame with test results.')           
# add the plots to the geodataframe of middle schools subject results 
plotsDFs = pd.DataFrame(plots, columns=['DBN', 'Dvst_chart'])

schools_mappable_plots = pd.merge(schools_mappable_plots, plotsDFs, left_on = 'DBN', right_on='DBN')
    
del schoolDFs, columns_to_plot, plotsDFs
print('Done.')   

Making test results plots ...


100%|██████████| 455/455 [00:36<00:00, 12.54it/s]

Adding plots to the data frame with test results.
Done.


In [67]:
schools_mappable_plots.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 455 entries, 0 to 454
Data columns (total 52 columns):
 #   Column                         Non-Null Count  Dtype   
---  ------                         --------------  -----   
 0   OBJECTID                       455 non-null    int32   
 1   LEGAL_NAME                     455 non-null    object  
 2   PHYSADDRLINE1                  455 non-null    object  
 3   PHYSCITY                       455 non-null    object  
 4   COUNTY_DESC                    455 non-null    object  
 5   RECORD_TYPE_DESC               455 non-null    object  
 6   SDL_DESC                       455 non-null    object  
 7   geometry                       455 non-null    geometry
 8   DBN                            455 non-null    object  
 9   School Name                    455 non-null    object  
 10  12yrs avg Lvl 1 Math           454 non-null    float64 
 11  12yrs avg Lvl 2 Math           454 non-null    float64 
 12  12yrs avg Lvl 3 Math        

In [68]:
# Make plots for popups in the map and add them as columns to the mappable dataframe

# Set interactive mode off
plt.ioff()

# list of schools names

schoolsNames = schools_mappable_plots['DBN'].to_list()
testResults = allResultsDF

# Create disctionnary to hold the dataframes by schools
schoolDFs = {}

# Make dataframes by schools 
for name in schoolsNames:
    dfName = name
    schoolDFs[dfName] = testResults[testResults['DBN'] == name]

plots = []
plotsDFs = {}

print("Making test results plots ...")

for subject in subjects:
    columns_to_plot = [f"Level 4 {subject}", f"Level 3 {subject}", f"Level 2 {subject}", f"Level 1 {subject}"]  
    # Plot dataframes by school
    for schoolDF, current_dataframe in tqdm(schoolDFs.items()):
        # schoolDF contains the name of the dataframe
        # current_dataframe contains the dataframe itself

            # Do something with current_dataframe
            # Create a plot
            fig = create_plot(current_dataframe, schoolDF, columns_to_plot)

            # Convert the plot to a PNG image and then encode it
            io_buf = BytesIO()
            fig.savefig(io_buf, format='png', bbox_inches='tight')
            # Close the figure
            plt.close()
            #Reading file to get the base64 string
            io_buf.seek(0)
            base64_string = base64.b64encode(io_buf.read()).decode('utf8')

            pair = (schoolDF, base64_string)

            plots.append(pair)

    # add the plots to the geodataframe of middle schools subject results 
    plotsDF = pd.DataFrame(plots, columns=['DBN', f'plot {subject}'])

    plotsDFs[subject] = plotsDF
    
print('Adding plots to the data frame with test results.')                
for subject, df in plotsDFs.items():
    schools_mappable_plots = pd.merge(schools_mappable_plots, df, left_on = 'DBN', right_on='DBN')

print('Done.')     

Making test results plots ...


100%|██████████| 455/455 [01:42<00:00,  4.44it/s]

Adding plots to the data frame with test results.
Done.


In [69]:
## Saving the resulting geodataframe into geoJSON file to make a map separately.

# If the area to display is less than the whole city or the number of schools
# selected to display is relatively small, the map can be displayed within a jupyter notebook,
# but in this case the dataframe is too big and the map is too loaded with symbols to use them this way.
# Therefore, we'll separate the map making and the data analysis into different notebooks and 
# later save a maps as html file. The geoJSON is used at this next step.

fname = 'schoolDataPlots2013-2025.geojson'
fpath = os.path.join(basePath, outputFolder, fname)
print(f'Saving to {fpath} ...')
schools_mappable_plots.to_file(fpath, driver="GeoJSON")
print('Saved.')

del fname, fpath

Saving to G:\My Drive\Kids\NYC_schools_mapped\processed_data\schoolDataPlots2013-2025.geojson ...
Saved.


In [70]:
schools_mappable_plots.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 910 entries, 0 to 909
Data columns (total 54 columns):
 #   Column                         Non-Null Count  Dtype   
---  ------                         --------------  -----   
 0   OBJECTID                       910 non-null    int32   
 1   LEGAL_NAME                     910 non-null    object  
 2   PHYSADDRLINE1                  910 non-null    object  
 3   PHYSCITY                       910 non-null    object  
 4   COUNTY_DESC                    910 non-null    object  
 5   RECORD_TYPE_DESC               910 non-null    object  
 6   SDL_DESC                       910 non-null    object  
 7   geometry                       910 non-null    geometry
 8   DBN                            910 non-null    object  
 9   School Name                    910 non-null    object  
 10  12yrs avg Lvl 1 Math           908 non-null    float64 
 11  12yrs avg Lvl 2 Math           908 non-null    float64 
 12  12yrs avg Lvl 3 Math        